In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
train = pd.read_csv("C:/Users/eduri/OneDrive/Documents/Semestre IV/DPII/archive_spacy/train.csv")

In [2]:
test = pd.read_csv("C:/Users/eduri/OneDrive/Documents/Semestre IV/DPII/archive_spacy/test.csv")

In [3]:
test

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."
...,...,...
17192,49155,thought factory: left-right polarisation! #tru...
17193,49156,feeling like a mermaid ð #hairflip #neverre...
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...
17195,49158,"happy, at work conference: right mindset leads..."


In [5]:
train

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [6]:
label = train['label']

In [7]:
train.drop(['label'], axis =1, inplace = True)
train

,id,tweet
0,1,@user when a father is dysfunctional and is s...
1,2,@user @user thanks for #lyft credit i can't us...
2,3,bihday your majesty
3,4,#model i love u take with u all the time in ...
4,5,factsguide: society now #motivation
...,...,...
31957,31958,ate @user isz that youuu?ðððððð...
31958,31959,to see nina turner on the airwaves trying to...
31959,31960,listening to sad songs on a monday morning otw...
31960,31961,"@user #sikh #temple vandalised in in #calgary,..."


In [8]:
combi = train.append(test)
combi

C:\Users\eduri\AppData\Local\Temp\ipykernel_42548\3060607916.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combi = train.append(test)


,id,tweet
0,1,@user when a father is dysfunctional and is s...
1,2,@user @user thanks for #lyft credit i can't us...
2,3,bihday your majesty
3,4,#model i love u take with u all the time in ...
4,5,factsguide: society now #motivation
...,...,...
17192,49155,thought factory: left-right polarisation! #tru...
17193,49156,feeling like a mermaid ð #hairflip #neverre...
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...
17195,49158,"happy, at work conference: right mindset leads..."


In [9]:
tweets = train['tweet']
count_words = tweets.str.findall(r'(\w+)').str.len()
print(count_words.sum())

443676


In [10]:
import re
from nltk.corpus import stopwords

tweets = tweets.str.lower()

tweets.apply(lambda x: re.sub("[^a-z\s]","",x))

tweets = tweets.str.replace("#", " ")

tweets = tweets.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

stopwords = set(stopwords.words("english"))

tweets = tweets.apply(lambda x: ' '.join(word for word in x.split() if word not in stopwords))

In [11]:
most_freq_words = pd.Series(' '.join(tweets).lower().split()).value_counts()[:25]

tweets = tweets.apply(lambda x: " ".join(word for word in x.split() if word not in most_freq_words))

print(most_freq_words)

count_words = tweets.str.findall(r'(\w+)').str.len()
print(count_words.sum())

@user       17297
love         2640
day          1689
&amp;        1580
happy        1437
like          994
i'm           993
â¦           980
new           972
time          959
thankful      940
positive      920
get           906
life          896
good          802
bihday        778
people        749
see           738
can't         729
one           697
smile         664
want          616
today         612
take          612
healthy       590
dtype: int64
259660


In [12]:
from collections import Counter
from itertools import chain

v = tweets.str.split().tolist()
c = Counter(chain.from_iterable(v))

tweets = [' '.join([j for j in i if c[j] > 1]) for i in v]

total_word = 0
for x, word in enumerate(tweets):
    num_word = len(word.split())
    total_word = total_word + num_word
print(total_word)

195938


In [13]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load('en_core_web_lg')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [14]:
import spacy
import en_core_web_lg

nlp = en_core_web_lg.load()
document= nlp(tweets[0])
print("Document : ", document)
print("Tokens : ")
for  token in document:
    print(token.text)

Document :  father selfish drags kids dysfunction. run
Tokens : 
father
selfish
drags
kids
dysfunction
.
run


In [15]:
document = nlp(tweets[0])
print(document)
for token in document:
    print(token.text, token.vector.shape)

father selfish drags kids dysfunction. run
father (300,)
selfish (300,)
drags (300,)
kids (300,)
dysfunction (300,)
. (300,)
run (300,)


In [16]:
document = nlp.pipe(tweets)
tweets_vector = np.array([tweet.vector for tweet in document])
print(tweets_vector.shape)

(31962, 300)


In [18]:
X = tweets_vector
y = label

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y, stratify=y,
                                                 test_size=0.3, random_state=1)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((22373, 300), (9589, 300), (22373,), (9589,))

In [21]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=10, max_iter=1000).fit(X_train,y_train)
print(model.score(X_train, y_train))

0.9468555848567469


In [22]:
y_pred = model.predict(X_val)
print(model.score(X_val, y_val))

0.9414954635519867


In [23]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_val, y_pred))

[[8769  147]
 [ 414  259]]
